## 해외축구 경기 정보
- 관심있는 축구 경기 정보 받기
- 보고싶은 경기를 시청할 수 없을 때, 스코어에 변화가 생길떄마다 현재 경기 스코어와 득점 정보를 전송
- 보고싶은 경기가 진행되는 카테고리(컵, 리그)를 먼저 정해야 함
- (경기 시작 전) 라인업 정보가 뜨면 각 팀의 선발 명단 크롤링
- (경기 종료 후) 최종 스코어와 세부 기록 크롤링

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import json
import numpy as np
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [183]:
# 1. 카테고리 선택하기
category = ['epl', 'primera', 'bundesliga', 'seria', 'ligue1',
            'denmark', 'champs', 'europa', 'uecl', 'facup',
            'carlingcup', 'copadelrey', 'clubworldcup']

cate = input("카테고리를 선택하시오: ")
schedule_url = f'https://sports.news.naver.com/wfootball/schedule/index?category={cate}'



# 2. (경기 시작 전) 선발 라인업 정보 가져오기
# lineup_url = 'https://m.sports.naver.com/game/2023081465041892243/lineup'
lineup_url = 'https://api-gw.sports.naver.com/schedule/games/2023050110034152359/lineup'
response_2 = requests.get(lineup_url, headers=headers)
soup_2 = json.loads(response_2.text)

# 홈팀 라인업데이터
h_lineup_data = soup_2['result']['lineUpData']['lineup']['home']['players']['lineup']

# 홈팀 포메이션
h_formation = soup_2['result']['lineUpData']['lineup']['home']['formation'].split('-')

# 홈팀 라인업
home_start = []
for i in range(1):
    home_start.append(h_lineup_data[0][i]['name'])
for i in range(4):
    home_start.append(h_lineup_data[1][i]['name'])
for i in range(3):
    home_start.append(h_lineup_data[2][i]['name'])
for i in range(3):
    home_start.append(h_lineup_data[3][i]['name'])

# 어웨이팀 라인업데이터
a_lineup_data = soup_2['result']['lineUpData']['lineup']['away']['players']['lineup']

# 어웨이팀 포메이션
a_formation = soup_2['result']['lineUpData']['lineup']['away']['formation'].split('-')

# 어웨이팀 라인업
away_start = []
for i in range(1):
    away_start.append(a_lineup_data[0][i]['name'])
for i in range(2):
    away_start.append(a_lineup_data[1][i]['name'])
for i in range(4):
    away_start.append(a_lineup_data[2][i]['name'])
for i in range(3):
    away_start.append(a_lineup_data[3][i]['name'])
for i in range(1):
    away_start.append(a_lineup_data[4][i]['name'])

# 홈, 어웨이 선발명단
starting_player = {
    'Home':home_start,
    'Away':away_start[::-1]
}
df_starting_player = pd.DataFrame(starting_player)



# 3. (경기 중) 스코어에 변화가 생길 때마다 현재 경기 스코어와 득점 정보 가져오기



# 4. (경기 종료 후) 최종 스코어와 세부 기록 가져오기
record_url = 'https://api-gw.sports.naver.com/schedule/games/2023050110034152359'
response_4 = requests.get(record_url, headers=headers)
soup_4 = json.loads(response_4.text)

# 최종 스코어
result_data = soup_4['result']['game']
# print(result_data['homeTeamName'], result_data['homeTeamScore'], result_data['awayTeamName'], result_data['awayTeamScore'])

# 홈팀이 득점한 시간
home_t = list(t['time'] for t in result_data['scorers']['home'])
home_at = list(a_t['addedTime'] for a_t in result_data['scorers']['home'])

# 홈팀 득점한 선수
home_player = list(p['playerName'] for p in result_data['scorers']['home'])

# 어웨이팀이 득점한 시간
away_t = list(t['time'] for t in result_data['scorers']['away'])
away_at = list(a_t['addedTime'] for a_t in result_data['scorers']['away'])

# 어웨이팀 득점한 선수
away_player = list(p['playerName'] for p in result_data['scorers']['away'])

# 홈팀 경기 정보
home_data = {
    'Name':home_player,
    't':home_t,
    'at':home_at
}
df_h = pd.DataFrame(home_data)
HomeFinalInfo = df_h.fillna(0)
HomeFinalInfo["Time"] = HomeFinalInfo['at'] + HomeFinalInfo['t']
# print(HomeFinalInfo)

# 어웨이팀 경기 정보
away_data = {
    'Name':away_player,
    't':away_t,
    'at':away_at
}
df_a = pd.DataFrame(away_data)
AwayFinalInfo = df_a.fillna(0)
AwayFinalInfo["Time"] = AwayFinalInfo['at'] + AwayFinalInfo['t']
# print(AwayFinalInfo)

In [16]:
# 1.
category = ['epl', 'primera', 'bundesliga', 'seria', 'ligue1',
            'denmark', 'champs', 'europa', 'uecl', 'facup',
            'carlingcup', 'copadelrey', 'clubworldcup']

cate = input("카테고리를 선택하시오: ")
schedule_url = f'https://sports.news.naver.com/wfootball/schedule/index?category={cate}'

response_1 = requests.get(schedule_url, headers=headers)
soup_1 = bs(response_1.text, 'html.parser')

In [24]:
soup_1.select('script')[-5]

# new naver.sports ~~ ); 끝나는 텍스트 가져오기
# 그 안에서 {} 안에 내용을 가져오기
# json.loads로 파이썬 딕셔너리로 변경

<script type="text/javascript">
	new naver.sports.schedule.SchedulePage({
		template: {
			seasonListTemplate : "_wfootballSeasonListTemplate"
		},
		today: '20230802',
		defaultDateId: '20230812',
		section: 'wfootball',
		category: 'epl',
		seasonList: [{"startYear":"2023","title":"2023/24","seasonName":"2023-24","yearMonthList":["202308","202309","202310","202311","202312","202401","202402","202403","202404","202405"],"yearMonthModelList":[{"year":"2023","month":"01","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"02","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"03","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"04","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"05","

In [68]:
soup_1_1 = soup_1.select('script')[-5]
# soup_1_1.select('')

In [69]:
soup_1.select('#container > script:nth-child(21)')

[<script type="text/javascript">
 	new naver.sports.schedule.SchedulePage({
 		template: {
 			seasonListTemplate : "_wfootballSeasonListTemplate"
 		},
 		today: '20230802',
 		defaultDateId: '20230812',
 		section: 'wfootball',
 		category: 'epl',
 		seasonList: [{"startYear":"2023","title":"2023/24","seasonName":"2023-24","yearMonthList":["202308","202309","202310","202311","202312","202401","202402","202403","202404","202405"],"yearMonthModelList":[{"year":"2023","month":"01","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"02","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"03","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"04","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","mon

In [73]:
soup_1_1.text

'\n\tnew naver.sports.schedule.SchedulePage({\n\t\ttemplate: {\n\t\t\tseasonListTemplate : "_wfootballSeasonListTemplate"\n\t\t},\n\t\ttoday: \'20230802\',\n\t\tdefaultDateId: \'20230812\',\n\t\tsection: \'wfootball\',\n\t\tcategory: \'epl\',\n\t\tseasonList: [{"startYear":"2023","title":"2023/24","seasonName":"2023-24","yearMonthList":["202308","202309","202310","202311","202312","202401","202402","202403","202404","202405"],"yearMonthModelList":[{"year":"2023","month":"01","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"02","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"03","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"04","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"

In [116]:
import re
print(re.search(r"{.*}", soup_1_1.text))

<re.Match object; span=(225, 43467), match='{"startYear":"2023","title":"2023/24","seasonName>


In [71]:
# calendar = json.loads(re.search(r"\(({.*})\)", soup_1_1.text).group(1))

In [54]:
print(re.search(r"\(({.*})\)", soup_1_1.text))

None


In [114]:
re.search(r"\{.*\}", soup_1_1.text).group(0)

'{"startYear":"2023","title":"2023/24","seasonName":"2023-24","yearMonthList":["202308","202309","202310","202311","202312","202401","202402","202403","202404","202405"],"yearMonthModelList":[{"year":"2023","month":"01","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"02","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"03","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"04","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"05","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month":"06","enable":false,"leagueTitle":null,"leagueShotTitle":null,"firstHalfColor":null,"secondHalfColor":null},{"year":"2023","month"

In [122]:
# 2. (경기 시작 전) 선발 라인업 정보 가져오기
# lineup_url = 'https://m.sports.naver.com/game/2023081465041892243/lineup'
lineup_url = 'https://api-gw.sports.naver.com/schedule/games/2023050110034152359/lineup'
response_2 = requests.get(lineup_url, headers=headers)
soup_2 = json.loads(response_2.text)

In [134]:
h_lineup_data = soup_2['result']['lineUpData']['lineup']['home']['players']['lineup']

In [154]:
soup_2['result']['lineUpData']['lineup']['home']['formation'].split('-')

['4', '3', '3']

In [151]:
len(soup_2['result']['lineUpData']['lineup']['home']['formation'].split('-'))

3

In [160]:
print(h_lineup_data[0][0]['name'])

알리송


In [177]:
home_start = []
for i in range(1):
    home_start.append(h_lineup_data[0][i]['name'])
for i in range(4):
    home_start.append(h_lineup_data[1][i]['name'])
for i in range(3):
    home_start.append(h_lineup_data[2][i]['name'])
for i in range(3):
    home_start.append(h_lineup_data[3][i]['name'])

home_start

['알리송',
 '알렉산더 아놀드',
 '코나테',
 '반 다이크',
 '로버트슨',
 '엘리엇',
 '파비뉴',
 '존스',
 '살라',
 '각포',
 '디아스']

In [163]:
a_lineup_data = soup_2['result']['lineUpData']['lineup']['away']['players']['lineup']

In [164]:
soup_2['result']['lineUpData']['lineup']['away']['formation'].split('-')

['3', '4', '2', '1']

In [176]:
away_start = []

for i in range(1):
    # print(a_lineup_data[0][i]['name'])
    away_start.append(a_lineup_data[0][i]['name'])
for i in range(2):
    away_start.append(a_lineup_data[1][i]['name'])
for i in range(4):
    away_start.append(a_lineup_data[2][i]['name'])
for i in range(3):
    away_start.append(a_lineup_data[3][i]['name'])
for i in range(1):
    away_start.append(a_lineup_data[4][i]['name'])

away_start[::-1]

['포스터',
 '로메로',
 '다이어',
 '데이비스',
 '포로',
 '스킵',
 '호이비에르',
 '페리시치',
 '쿨루셰프스키',
 '손흥민',
 '케인']

In [182]:
starting_player = {
    'Home':home_start,
    'Away':away_start[::-1]
}

df_starting_player = pd.DataFrame(starting_player)
df_starting_player

,Home,Away
0,알리송,포스터
1,알렉산더 아놀드,로메로
2,코나테,다이어
3,반 다이크,데이비스
4,로버트슨,포로
5,엘리엇,스킵
6,파비뉴,호이비에르
7,존스,페리시치
8,살라,쿨루셰프스키
9,각포,손흥민


In [3]:
# 4.
record_url = 'https://api-gw.sports.naver.com/schedule/games/2023050110034152359'
response_4 = requests.get(record_url, headers=headers)
# print(res.text)
soup_4 = json.loads(response_4.text)


In [4]:
print(soup_4)

{'code': 200, 'success': True, 'result': {'game': {'gameId': '2023050110034152359', 'superCategoryId': 'football', 'upperCategoryId': 'wfootball', 'categoryId': 'epl', 'categoryName': '프리미어리그', 'gameDate': '2023-05-01', 'gameDateTime': '2023-05-01T00:30:00', 'stadium': 'Anfield', 'title': '', 'homeTeamCode': '9', 'homeTeamName': '리버풀', 'homeTeamScore': 4, 'awayTeamCode': '19', 'awayTeamName': '토트넘', 'awayTeamScore': 3, 'winner': 'HOME', 'statusCode': 'RESULT', 'statusNum': 4, 'statusInfo': '경기종료', 'cancel': False, 'suspended': False, 'hasVideo': True, 'seasonCode': None, 'seasonYear': 2022, 'generalTitle': None, 'generalInfo3': None, 'manualRelayUrl': None, 'emptyScoreBeforeResult': False, 'reversedHomeAway': False, 'homeTeamEmblemUrl': 'https://sports-phinf.pstatic.net/team/wfootball/default/9.png', 'awayTeamEmblemUrl': 'https://sports-phinf.pstatic.net/team/wfootball/default/19.png', 'homeTeamShortName': '리버풀', 'awayTeamShortName': '토트넘', 'homeTeamFullName': '리버풀 FC', 'awayTeamFullNa

In [6]:
result_data = soup_4['result']['game']
result_data

{'gameId': '2023050110034152359',
 'superCategoryId': 'football',
 'upperCategoryId': 'wfootball',
 'categoryId': 'epl',
 'categoryName': '프리미어리그',
 'gameDate': '2023-05-01',
 'gameDateTime': '2023-05-01T00:30:00',
 'stadium': 'Anfield',
 'title': '',
 'homeTeamCode': '9',
 'homeTeamName': '리버풀',
 'homeTeamScore': 4,
 'awayTeamCode': '19',
 'awayTeamName': '토트넘',
 'awayTeamScore': 3,
 'winner': 'HOME',
 'statusCode': 'RESULT',
 'statusNum': 4,
 'statusInfo': '경기종료',
 'cancel': False,
 'suspended': False,
 'hasVideo': True,
 'seasonCode': None,
 'seasonYear': 2022,
 'generalTitle': None,
 'generalInfo3': None,
 'manualRelayUrl': None,
 'emptyScoreBeforeResult': False,
 'reversedHomeAway': False,
 'homeTeamEmblemUrl': 'https://sports-phinf.pstatic.net/team/wfootball/default/9.png',
 'awayTeamEmblemUrl': 'https://sports-phinf.pstatic.net/team/wfootball/default/19.png',
 'homeTeamShortName': '리버풀',
 'awayTeamShortName': '토트넘',
 'homeTeamFullName': '리버풀 FC',
 'awayTeamFullName': '토트넘 홋스퍼 FC

In [7]:
# 최종 스코어
print(result_data['homeTeamName'], result_data['homeTeamScore'], result_data['awayTeamName'], result_data['awayTeamScore'])

리버풀 4 토트넘 3


In [8]:
result_data['scorers']['home'],result_data['scorers']['away']

([{'time': 3, 'addedTime': None, 'playerName': '존스', 'ownGoal': False},
  {'time': 5, 'addedTime': None, 'playerName': '디아스', 'ownGoal': False},
  {'time': 15, 'addedTime': None, 'playerName': '살라', 'ownGoal': False},
  {'time': 90, 'addedTime': 4, 'playerName': '조타', 'ownGoal': False}],
 [{'time': 40, 'addedTime': None, 'playerName': '케인', 'ownGoal': False},
  {'time': 77, 'addedTime': None, 'playerName': '손흥민', 'ownGoal': False},
  {'time': 90, 'addedTime': 3, 'playerName': '히샬리송', 'ownGoal': False}])

In [9]:
# 홈팀이 득점한 시간
home_t = list(t['time'] for t in result_data['scorers']['home'])
home_at = list(a_t['addedTime'] for a_t in result_data['scorers']['home'])

# 홈팀 득점한 선수
home_player = list(p['playerName'] for p in result_data['scorers']['home'])

# 어웨이팀이 득점한 시간
away_t = list(t['time'] for t in result_data['scorers']['away'])
away_at = list(a_t['addedTime'] for a_t in result_data['scorers']['away'])

# 어웨이팀 득점한 선수
away_player = list(p['playerName'] for p in result_data['scorers']['away'])

In [10]:
home_data = {
    'Name':home_player,
    't':home_t,
    'at':home_at
}
df_h = pd.DataFrame(home_data)
HomeFinalInfo = df_h.fillna(0)
HomeFinalInfo

,Name,t,at
0,존스,3,0.0
1,디아스,5,0.0
2,살라,15,0.0
3,조타,90,4.0


In [11]:
HomeFinalInfo["Time"] = HomeFinalInfo['at'] + HomeFinalInfo['t']
HomeFinalInfo

,Name,t,at,Time
0,존스,3,0.0,3.0
1,디아스,5,0.0,5.0
2,살라,15,0.0,15.0
3,조타,90,4.0,94.0


In [12]:
away_data = {
    'Name':away_player,
    't':away_t,
    'at':away_at
}
df_a = pd.DataFrame(away_data)
AwayFinalInfo = df_a.fillna(0)
AwayFinalInfo

,Name,t,at
0,케인,40,0.0
1,손흥민,77,0.0
2,히샬리송,90,3.0


In [13]:
AwayFinalInfo["Time"] = AwayFinalInfo['at'] + AwayFinalInfo['t']
AwayFinalInfo

,Name,t,at,Time
0,케인,40,0.0,40.0
1,손흥민,77,0.0,77.0
2,히샬리송,90,3.0,93.0


In [14]:
(AwayFinalInfo)

,Name,t,at,Time
0,케인,40,0.0,40.0
1,손흥민,77,0.0,77.0
2,히샬리송,90,3.0,93.0
